### Dependencias necesarias

In [37]:
import overpass
import geopandas
import matplotlib.pyplot as plt

### Obtener las calles de Iñas con la API de Overpass

Buscamos las calles/carreteras de titularidad municipal. Usamos la etiqueta de osm "highway" = "residential" y usamos formato de salida GeoJSON

In [38]:
api = overpass.API()
result = api.get("""
area["name"~"^Iñás$"];
way["highway" = "residential"](area);
(._;>;);
out geom;""", responseformat="geojson")

### Procesar el resultado de Overpass

Con geopandas obtenemos el geoDataFrame a partir del GeoJSON de Overpass. Ajustamos el crs para hacerlo compatible con el mapa base que vamos a usar después. Primero tenemos que asignarle en el que vienen los datos de OSM (4326), y luego lo cambiamos al 3857 (Web Mercator), que es el estándar de facto en aplicaciones web de mapas.

In [39]:
streetsDataFrame = geopandas.GeoDataFrame.from_features(result['features'])
streetsDataFrame.crs = "EPSG:4326"
streetsDataFrame=streetsDataFrame.to_crs(epsg=3857)

In [ ]:
streetsDataFrame.columns

### Creamos la visualización del mapa

Primero importamos contextily, que es un paquete Python que nos permite usar mapas base habituales en las aplicaciones web de mapas. Específicamente nos permite usar esos mapas base con matplotlib, que es una librería para generar gráficos y figuras, pero que también es capaz de generar mapas estáticos.

Aquí vamos a jugar básicamente con opciones de matplotlib para generar un mapa estático con nuestros datos, crear una leyenda para ver las calles por colores, en base a su estado, y ponerle un título al mapa.

In [ ]:
import contextily as ctx

fig, ax = plt.subplots(figsize=(50, 50))
colors = {'good': 'green', 'bad': 'red', 'intermediate': 'orange', 'very_bad': 'black'}

# Filtramos solo las calles que tienen valor de smoothness
streets_with_smoothness = streetsDataFrame.dropna(subset=['smoothness'])

for state, data in streets_with_smoothness.groupby('smoothness'):
    if state in colors:  # Solo plotear si tenemos un color definido
        color = colors[state]
        data.plot(
            color=color,
            ax=ax,
            label=state,
            linewidth=2
        )

# Añadir mapa base
ctx.add_basemap(ax, 
                source=ctx.providers.CartoDB.Positron,
                zoom=15)

plt.title("Estado del pavimento en las calles de Iñás", 
          pad=20,  # Espacio entre el título y el gráfico
          fontsize=40,  # Tamaño de la fuente
          fontweight='bold')  # Negrita
ax.legend(bbox_to_anchor=(1.0, .5), prop={'size': 28})

plt.show()